# Digit Recognizer and ConvNet

## Imports

In [86]:
import pandas as pd
import numpy as np

import keras as K
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from sklearn.model_selection import train_test_split

## Load train and test set

In [87]:
train_data = pd.read_csv('./data/train.csv')
train_data.head() 

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Data needs to be modified to be able to fit into ConvNet. 
Instead of pixels beeing represented in rows, ConvNet needs them in matrix. For example [28 x 28].

In [88]:
#reshape train data
df_X = train_data.iloc[:,1:].values.reshape(len(train_data),28,28,1)
df_y = train_data.iloc[:,0].values

In [89]:
# convert labels tocategorical features. 
# as now the numbers hold numerical relationship, that we do not need.
df_y = K.utils.to_categorical(df_y, num_classes=10)


In [90]:
df_X = np.array(df_X)
df_X.shape

(42000, 28, 28, 1)

In [91]:
df_y = np.array(df_y)
df_y.shape


(42000, 10)

In [92]:
# split the data
X_train, X_test, y_train,y_test = train_test_split(df_X,df_y, test_size=0.2,random_state=0)

## Create CNN model


In [99]:
model = Sequential()
# first Conv layer with 28x28 as input
# 32 3x3 filters (+ ReLU)
model.add(Convolution2D(32,3,input_shape=(28,28,1), activation='relu'))
model.add(Convolution2D(32,3, activation='relu'))



In [100]:
# MaxPooling layer
model.add(MaxPooling2D(pool_size=(2,2)))


In [101]:
model.add(Convolution2D(64,3, activation='relu'))
model.add(Convolution2D(64,3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [102]:
# Flatten Nodes
model.add(Flatten())


In [103]:
#Fully  connected layer 512 nodes
model.add(Dense(512))


In [104]:
# Ouput layer with 10 nodes
model.add(Dropout(0.5)) #helps avoid overfitting
model.add(Dense(10))
model.add(Activation('softmax'))


In [105]:
# compile with calculaing errors by defining loss function
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [76]:
# add softmax activation layer
# model.add(Activation('softmax'))

In [106]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_9 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 10, 10, 64)        18496     
                                                                 
 conv2d_11 (Conv2D)          (None, 8, 8, 64)          36928     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 4, 4, 64)         0         
 2D)                                                  

## Fit the model

In [107]:
# for testing fit with 100 sample images
model.fit(X_train, y_train,validation_data=(X_test, y_test),batch_size=64, epochs=1)

525/525 [==============================] - 81s 153ms/step - loss: 0.6678 - accuracy: 0.9035 - val_loss: 0.1074 - val_accuracy: 0.9658


In [83]:
model.predict(X_test)

263/263 [==============================] - 2s 7ms/step


array([[1.5765404e-26, 0.0000000e+00, 4.6972809e-26, ..., 2.0965206e-32,
        9.9999988e-01, 4.7851275e-32],
       [2.1861805e-34, 0.0000000e+00, 2.7861673e-13, ..., 1.6611608e-27,
        2.3934017e-29, 3.4224869e-27],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.1878241e-03,
        2.4432188e-22, 9.9881220e-01],
       ...,
       [7.0014995e-24, 1.1437653e-33, 1.0000000e+00, ..., 0.0000000e+00,
        4.7180195e-20, 0.0000000e+00],
       [3.4834156e-29, 0.0000000e+00, 9.2747257e-34, ..., 1.0000000e+00,
        1.5358199e-33, 4.1708156e-16],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [108]:
model.evaluate(X_test,y_test)

263/263 [==============================] - 4s 14ms/step - loss: 0.1074 - accuracy: 0.9658


[0.1074395626783371, 0.965833306312561]

In [109]:
model.save('./model/')

2023-05-17 12:10:29.863257: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,512]
	 [[{{node inputs}}]]
2023-05-17 12:10:30.052687: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,512]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: ./results/assets


INFO:tensorflow:Assets written to: ./results/assets
